In [ ]:
# ----------------------------
# 0. (Colab) Mount Google Drive
# ----------------------------
COLAB = False  # set False if running locally
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.


In [4]:
# ============================================================
# MSML 612 – UMD Sagemakers
# Ateeq Ur Rehman: Feature Engineering + Baseline Modeling
# (Weather removed)
# ============================================================
# 1. Load daily data & ensure continuous index
# 2. Engineer temporal, event, agency-share features
# 3. Build supervised targets for 1,3,7-day horizons
# 4. Train/evaluate Linear, Lasso, RandomForest, SARIMAX
# 5. Save metrics, feature importances, plots, data dictionary
# ============================================================


# --------------------
# 1. Imports & Config
# --------------------
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
# import warnings
# warnings.filterwarnings('ignore')
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.filterwarnings("ignore", category=ValueWarning)

from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

from statsmodels.tsa.statespace.sarimax import SARIMAX

# --------- CONFIG ---------
DATA_PATH   = Path("/content/drive/MyDrive/MSML612 Project/Data")
OUTPUT_PATH = Path("/content/drive/MyDrive/MSML612 Project/Outputs_Ateeq")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

DAILY_FILE  = DATA_PATH / "daily_timeseries_parking_violations_v2.csv"

TRAIN_END = pd.Timestamp("2024-12-31")
VAL_END   = pd.Timestamp("2025-03-31")  # test = Apr–May 2025

TARGET    = "num_violations"
DATE_COL  = "date"
HORIZONS  = [1, 3, 7]
# --------------------------

In [5]:
# ----------------------
# 2. Utility Functions
# ----------------------
def load_and_patch_daily(path: Path, date_col="date", target_col="num_violations") -> pd.DataFrame:
    """
    Load daily series, reindex to continuous daily range, safely cast int-like cols.
    """
    df = pd.read_csv(path, parse_dates=[date_col])

    full_index = pd.date_range(df[date_col].min(), df[date_col].max(), freq="D")
    df = df.set_index(date_col).reindex(full_index)
    df.index.name = date_col
    df.reset_index(inplace=True)

    # Fill numeric columns with 0
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c]) and c != date_col]
    df[num_cols] = df[num_cols].fillna(0)

    # Calendar features
    df["year"]        = df[date_col].dt.year
    df["month"]       = df[date_col].dt.month
    df["day_of_week"] = df[date_col].dt.day_name()
    df["is_weekend"]  = (df[date_col].dt.dayofweek >= 5).astype("int8")

    # Holiday column may exist & contain NaN
    if "is_holiday" in df.columns:
        df["is_holiday"] = pd.to_numeric(df["is_holiday"], errors="coerce").fillna(0).astype("int8")
    else:
        df["is_holiday"] = 0

    return df


def add_temporal_features(df: pd.DataFrame,
                          date_col=DATE_COL,
                          target_col=TARGET) -> pd.DataFrame:
    """
    Add lags, rolling stats, Fourier seasonality, Cherry Blossom flag,
    agency shares, and DOW one-hots.
    """
    df = df.copy().sort_values(date_col)

    # Lags
    for lag in [1, 7, 14, 30]:
        df[f"{target_col}_lag_{lag}"] = df[target_col].shift(lag)

    # Rolling stats
    df[f"{target_col}_roll7_mean"]  = df[target_col].rolling(7).mean()
    df[f"{target_col}_roll30_mean"] = df[target_col].rolling(30).mean()
    df[f"{target_col}_roll30_std"]  = df[target_col].rolling(30).std()

    # Relative deviation vs 7-day mean
    df["roc_vs_roll7"] = (df[target_col] - df[f"{target_col}_roll7_mean"]) / (df[f"{target_col}_roll7_mean"] + 1e-6)

    # Fourier (annual) terms
    doy = df[date_col].dt.dayofyear
    for k in [1, 2]:
        df[f"sin_{k}"] = np.sin(2*np.pi*k*doy/365.25)
        df[f"cos_{k}"] = np.cos(2*np.pi*k*doy/365.25)

    # Cherry Blossom flag (Mar 15–Apr 20)
    def cherry_flag(d):
        start = pd.Timestamp(year=d.year, month=3, day=15)
        end   = pd.Timestamp(year=d.year, month=4, day=20)
        return int(start <= d <= end)
    df["is_cherry_blossom"] = df[date_col].apply(cherry_flag)

    # Agency shares
    agency_cols = ["DDOT","DPW","MPD-1D","MPD-3D","OTHER","USCP"]
    existing_agencies = [c for c in agency_cols if c in df.columns]
    if existing_agencies:
        tot = df[existing_agencies].sum(axis=1).replace(0, np.nan)
        for c in existing_agencies:
            df[f"{c}_share"] = (df[c] / tot).fillna(0)

    # Day-of-week one-hot
    dummies = pd.get_dummies(df["day_of_week"], prefix="dow", drop_first=True)
    df = pd.concat([df, dummies], axis=1)

    return df


def make_supervised(df: pd.DataFrame, horizons, target=TARGET, date_col=DATE_COL) -> pd.DataFrame:
    """Create future target columns y_t_plus_h for each horizon."""
    df = df.copy()
    for h in horizons:
        df[f"y_t_plus_{h}"] = df[target].shift(-h)
    return df.dropna().reset_index(drop=True)


def chrono_split(df: pd.DataFrame, date_col, train_end, val_end):
    """Train <= train_end, Val (train_end,val_end], Test > val_end"""
    train = df[df[date_col] <= train_end]
    val   = df[(df[date_col] > train_end) & (df[date_col] <= val_end)]
    test  = df[df[date_col] > val_end]
    return train, val, test


def evaluate(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    mae = mean_absolute_error(y_true, y_pred)

    # Older sklearn: mean_squared_error has no `squared` kwarg, so we compute RMSE manually
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    # Safe MAPE (avoid divide-by-zero when counts are 0)
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1))) * 100

    return {"MAE": mae, "RMSE": rmse, "MAPE": mape}


def plot_pred_vs_actual(dates, y_true, y_pred, title, save_path):
    plt.figure(figsize=(11, 4))
    plt.plot(dates, y_true, label="Actual", linewidth=2)
    plt.plot(dates, y_pred, label="Predicted", linestyle='--', linewidth=2)
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Violations")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.close()


def plot_residual_hist(residuals, save_path, bins=20):
    plt.figure()
    plt.hist(residuals, bins=bins)
    plt.xlabel("Residual")
    plt.ylabel("Count")
    plt.title("Residual Distribution")
    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.close()



In [6]:
# ---------------------------------
# 3. Load & Feature Engineering
# ---------------------------------
daily_df   = load_and_patch_daily(DAILY_FILE)
feature_df = add_temporal_features(daily_df)

# Drop rows with NA from lags/rollings
feature_df = feature_df.dropna().reset_index(drop=True)

# Supervised dataset
supervised = make_supervised(feature_df, HORIZONS, target=TARGET, date_col=DATE_COL)

In [7]:

# ---------------------------------
# 4. Chronological Split
# ---------------------------------
train_df, val_df, test_df = chrono_split(supervised, DATE_COL, TRAIN_END, VAL_END)

# Build X (exclude target, future targets, and raw day_of_week text)
all_targets = [f"y_t_plus_{h}" for h in HORIZONS]
drop_cols   = [TARGET, DATE_COL, "day_of_week"] + all_targets
X_cols      = [c for c in supervised.columns if c not in drop_cols]

data_by_h = {}
for h in HORIZONS:
    y_col = f"y_t_plus_{h}"
    data_by_h[h] = {
        "X_train": train_df[X_cols], "y_train": train_df[y_col],
        "X_val":   val_df[X_cols],   "y_val":   val_df[y_col],
        "X_test":  test_df[X_cols],  "y_test":  test_df[y_col],
        "dates_test": test_df[DATE_COL].values
    }


In [8]:
# ---------------------------------
# 5. Baseline Models
# ---------------------------------
model_zoo = {
    "LinearRegression": Pipeline([("scaler", StandardScaler()),
                                  ("lr", LinearRegression())]),
    "LassoCV": Pipeline([("scaler", StandardScaler()),
                         ("lasso", LassoCV(cv=5, random_state=42, max_iter=10000))]),
    "RandomForest": RandomForestRegressor(n_estimators=500,
                                          random_state=42,
                                          n_jobs=-1,
                                          min_samples_split=5)
}

metrics_rows = []
pred_store   = {}

# Optional: silence the SARIMAX freq warning
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.filterwarnings("ignore", category=ValueWarning)

for h, data in data_by_h.items():
    pred_store[h] = {}
    X_train, y_train = data["X_train"], data["y_train"]
    X_test,  y_test  = data["X_test"],  data["y_test"]
    dates_test       = data["dates_test"]

    # -------------------------
    # 5a. Supervised baselines
    # -------------------------
    for name, model in model_zoo.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        m = evaluate(y_test, y_pred)
        m.update({"Model": name, "Horizon": h})
        metrics_rows.append(m)
        pred_store[h][name] = y_pred

    # -------------------------
    # 5b. SARIMAX baseline
    # (rolling refit version – consistent with earlier logic)
    # -------------------------
    ts_trainval = supervised.set_index(DATE_COL)[TARGET].asfreq('D').loc[:VAL_END]
    history = ts_trainval.copy()
    sar_preds = []

    for dt in dates_test:
        sar_model = SARIMAX(history,
                            order=(1,1,1),
                            seasonal_order=(1,1,1,7),
                            enforce_stationarity=False,
                            enforce_invertibility=False).fit(disp=False)
        forecast_val = sar_model.forecast(steps=h).iloc[-1]
        sar_preds.append(forecast_val)

        # append the true target for that dt so next iteration has updated history
        true_val = supervised.set_index(DATE_COL).loc[dt, TARGET]
        history  = pd.concat([history, pd.Series(true_val, index=[dt])]).asfreq('D')

    m = evaluate(y_test.values, np.array(sar_preds))
    m.update({"Model": "SARIMAX(1,1,1)(1,1,1,7)", "Horizon": h})
    metrics_rows.append(m)
    pred_store[h]["SARIMAX"] = np.array(sar_preds)

# -------------------------
# Metrics table
# -------------------------
metrics_df = pd.DataFrame(metrics_rows).set_index(["Horizon", "Model"]).sort_values(["Horizon","MAE"])
display(metrics_df)
metrics_df.to_csv(OUTPUT_PATH / "baseline_metrics_multi_horizon.csv")


MAE         RMSE        MAPE
Horizon Model                                                        
1       LinearRegression          561.722777   913.690395  122.245897
        LassoCV                   638.058911   930.002046  116.146006
        RandomForest              672.696442   916.719755  100.952923
        SARIMAX(1,1,1)(1,1,1,7)  1551.240624  1965.285336  192.470501
3       LinearRegression          644.387412   953.838202  247.746485
        RandomForest              695.542459  1001.231341  253.209741
        LassoCV                   709.532443  1006.348681  263.423839
        SARIMAX(1,1,1)(1,1,1,7)  1462.766386  1838.368950  206.171705
7       LassoCV                   610.831712  1018.853570  275.161073
        RandomForest              617.137958   966.428921  253.772607
        LinearRegression          653.808889  1036.689500  253.375263
        SARIMAX(1,1,1)(1,1,1,7)  1446.037310  1855.386495  217.787636

In [10]:

# ---------------------------------
# 6. Plots & Feature Importance
# ---------------------------------
for h, data in data_by_h.items():
    sub = metrics_df.loc[h]
    best_model = sub["MAE"].idxmin()
    y_test     = data["y_test"]
    y_pred_best = pred_store[h][best_model]
    dates      = data["dates_test"]

    plot_pred_vs_actual(dates, y_test.values, y_pred_best,
                        f"Pred vs Actual ({best_model}) – t+{h}",
                        OUTPUT_PATH / f"pred_vs_actual_best_h{h}.png")

    residuals = y_test.values - y_pred_best
    plot_residual_hist(residuals, OUTPUT_PATH / f"residual_hist_h{h}.png")

# RF feature importance (horizon=1)
rf = model_zoo["RandomForest"]
rf.fit(data_by_h[1]["X_train"], data_by_h[1]["y_train"])
fi = pd.DataFrame({"feature": X_cols, "importance": rf.feature_importances_}).sort_values("importance", ascending=False)
fi.to_csv(OUTPUT_PATH / "rf_feature_importance.csv", index=False)

plt.figure(figsize=(8,6))
plt.barh(fi.head(15)["feature"][::-1], fi.head(15)["importance"][::-1])
plt.xlabel("Importance")
plt.title("RandomForest Feature Importance (Top 15)")
plt.tight_layout()
plt.savefig(OUTPUT_PATH / "rf_feature_importance.png", dpi=200)
plt.close()


In [11]:

# ---------------------------------
# 7. Save features & Data Dictionary
# ---------------------------------
feature_df.to_csv(OUTPUT_PATH / "features_daily.csv", index=False)

data_dict = {
"date":"Date (daily granularity)",
"num_violations":"Total number of tickets issued that day (base target)",
"year":"Calendar year",
"month":"Calendar month number",
"is_holiday":"1 if US federal holiday, else 0",
"is_weekend":"1 if Saturday/Sunday, else 0",
"day_of_week":"Day name (categorical, one-hot later)",
"DDOT":"Tickets issued by DDOT that day",
"DPW":"Tickets issued by DPW that day",
"MPD-1D":"Tickets by MPD-1D",
"MPD-3D":"Tickets by MPD-3D",
"OTHER":"Tickets by other agencies",
"USCP":"Tickets by US Capitol Police",
"num_violations_lag_1":"Target value 1 day ago",
"num_violations_lag_7":"Target value 7 days ago",
"num_violations_lag_14":"Target value 14 days ago",
"num_violations_lag_30":"Target value 30 days ago",
"num_violations_roll7_mean":"7-day rolling mean of target",
"num_violations_roll30_mean":"30-day rolling mean of target",
"num_violations_roll30_std":"30-day rolling std dev of target",
"roc_vs_roll7":"Relative deviation from 7-day mean",
"sin_1":"1st harmonic sine term of day-of-year",
"cos_1":"1st harmonic cosine term of day-of-year",
"sin_2":"2nd harmonic sine term of day-of-year",
"cos_2":"2nd harmonic cosine term of day-of-year",
"is_cherry_blossom":"1 if Mar 15–Apr 20 (Cherry Blossom Festival window)",
"DDOT_share":"Proportion of DDOT tickets among agencies that day",
"DPW_share":"Proportion of DPW tickets that day",
"MPD-1D_share":"Proportion of MPD-1D tickets",
"MPD-3D_share":"Proportion of MPD-3D tickets",
"OTHER_share":"Proportion of OTHER tickets",
"USCP_share":"Proportion of USCP tickets",
"dow_*":"One-hot day-of-week indicators (Friday baseline dropped)",
"y_t_plus_1":"Target for 1-day-ahead forecasting",
"y_t_plus_3":"Target for 3-day-ahead forecasting",
"y_t_plus_7":"Target for 7-day-ahead forecasting"
}

pd.DataFrame(list(data_dict.items()), columns=["column","description"])\
  .to_csv(OUTPUT_PATH / "data_dictionary.csv", index=False)

print("DONE! Check", OUTPUT_PATH, "for CSVs, plots, and metrics.")


DONE! Check /content/drive/MyDrive/MSML612 Project/Outputs_Ateeq for CSVs, plots, and metrics.
